In [1]:
import requests
import re
#导入python-docx模块
import docx
from docx.oxml.ns import qn

In [ ]:
#创建一个空字典来装我们的标题、链接、发布日期
all_links = {} 

In [ ]:
#利用循环来构造网址
for i in range(1, 452, 45):
    start_url = 'https://jst.zj.gov.cn/'
    url = f'https://jst.zj.gov.cn/module/jpage/dataproxy.jsp?startrecord={i}&endrecord={i+44}&perpage=15'
    #最后一页只间隔了43，所有需要手动写
    if i == 451:
        url = f'https://jst.zj.gov.cn/module/jpage/dataproxy.jsp?startrecord={i}&endrecord=494&perpage=15'
    
    #将Form data写成字典，用于POST发送请求
    formdata = {
    'col': '1',
    'appid': '1',
    'webid': '3162',   
    'path': '%2F',
    'columnid': '1569296',
    'sourceContentType': '1',
    'unitid': '4803831',
    'webname': '%E6%B5%99%E6%B1%9F%E7%9C%81%E4%BD%8F%E6%88%BF%E5%92%8C%E5%9F%8E%E4%B9%A1%E5%BB%BA%E8%AE%BE%E5%8E%85',
    'permissiontype': '0'
    }
    #使用requests解析源代码
    html = requests.post(url, data = formdata).content.decode()
    #使用正则表达式获取文章链接
    links = re.findall('href=(.*?)title', html)

    #使用正则表达式获取标题
    titles = re.findall('title=(.*?)>', html)

    #使用正则表达式获取发布日期
    dates = re.findall('32px;">(.*?)<', html)
    
    for i in range (len(links)):
        http_link = (start_url + links[i]).replace('\'', '').strip()  #将链接与start_url拼接起来（这样才是正常的链接）
        title = titles[i].strip() 
        time = dates[i].replace('-    ', '').strip()
        all_links[title] = [http_link, time]  #结果存入all_links字典中

In [ ]:
def to_word(all_links):
    #构造请求头，模拟浏览器访问
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0"}
    
    counter = 0
    for title in all_links.keys():
        #创建新的word文档
        doc = docx.Document()

        #设置word文档字体样式等
        doc.styles['Normal'].font.name = u'宋体'
        doc.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
        #添加标题
        doc.add_heading(title, 1)
        
        #添加发布日期
        date = all_links[title][1]
        doc.add_paragraph(date)
        
        #添加正文
        page = requests.get(all_links[title][0], headers = header).content.decode('utf-8')
        articles = re.findall('2em;">(.*?)</p>', page, re.S)
        
        for article in articles:
            each = article.replace('<strong>', '').replace('</strong>', '').replace('<br/>', '').replace('<b>', '').replace('</b>', '')
            doc.add_paragraph(each)
        print(f"写入文章{title}。")
        doc.save(f'{title}.docx')
        
        counter += 1
        doc.add_page_break()
        
    print(f"共写入{counter}篇文章。")

In [ ]:
to_word(all_links)